In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import lognorm
import matplotlib.pyplot as plt
import shared

ve_or_ju = "ju"
year = 2019

In [ ]:
estimates = pd.read_json(f'web-lib/for_web_{ve_or_ju}_{year}.json', orient="records", convert_dates=["fin_time_median", "fin_time_start95", "fin_time_end95"])



In [ ]:
estimates.head().round(3)

In [ ]:
display(estimates.info())


In [ ]:
# Read the actual times after race and analyze estimates

results = pd.read_csv(f'data/results_with_dist_j{year}_{ve_or_ju}.tsv', delimiter="\t")
display(results.head().round(2))
results = results[["team-id", "leg-nro", "leg-time", "leg_distance"]]
results["leg-time"] = results["leg-time"] / 60
results = results.rename(index=str, columns={"team-id": "team_id", "leg-nro": "leg"})
results.head().round(3)

In [ ]:

with_result = pd.merge(estimates, results, how='left', on=['team_id', 'leg'])
with_result = with_result[np.isfinite(with_result["leg-time"])]

In [ ]:
with_result.head().round(3)


In [ ]:
with_result["ind_leg_time_estimate"] = np.exp(with_result["ind_log_mean"]) * with_result["leg_distance"]

In [ ]:
with_result["ind_error"] = np.abs(with_result["ind_leg_time_estimate"] - with_result["leg-time"])
with_result["ind_error_ratio"] = with_result["ind_error"] / with_result["leg-time"]
with_result["ind_interval_error"] = (with_result["personal_start95"] > with_result["leg-time"]) | (with_result["personal_end95"] < with_result["leg-time"])


In [ ]:
with_result.sort_values(by=['ind_error']).tail(10).round(2)

In [ ]:
reports = [f'Yksilöennusteen keskivirhe: {np.mean(with_result["ind_error"]).round(3)} minuuttia',
              f'Yksilöennusteen mediaanivirhe: {np.median(with_result["ind_error"]).round(3)} minuuttia',
           f'Yksilön aikaväliennuste väärin: {np.mean(with_result["ind_interval_error"]).round(4) * 100} %']

display(reports)  

In [ ]:
with_result["ind_interval"] = with_result["personal_end95"] - with_result["personal_start95"]


In [ ]:
ind_by_num_runs = with_result[["num_runs", "ind_interval", 'ind_error', 'ind_interval_error']].groupby('num_runs').agg({'ind_interval': ["mean", "median"], 'ind_error': ["mean", "median"], 'ind_interval_error': ["mean", "count"]}).round(2)
reports.append(ind_by_num_runs.to_string())
ind_by_num_runs

In [ ]:
with_result[["leg", 'ind_error', 'ind_interval_error', "num_runs"]].groupby('leg').agg(["mean", "median", "count"]).round(2)

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}
// # To disable auto-scrolling, execute this javascript in a notebook cell before other cells are executed:

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(16, 10))
plot = sns.scatterplot(x="team_id", y="ind_error", hue="num_runs", data=with_result)
plot.axes.set_ylim(0, with_result.ind_error.quantile(0.9))


In [ ]:
plt.figure(figsize=(16, 10))
plot = sns.scatterplot(x="team_id", y="leg-time", hue="ind_interval_error", alpha=0.4, data=with_result)
plot.axes.set_ylim(with_result["leg-time"].min() - 5, with_result["leg-time"].quantile(0.95))

In [ ]:
plt.figure(figsize=(16, 10))
plot = sns.scatterplot(x="leg-time", y="ind_error", hue="ind_interval_error", data=with_result)
plot.axes.set_ylim(0, with_result.ind_error.quantile(0.95))
plot.axes.set_xlim(with_result["leg-time"].min() - 5, with_result["leg-time"].quantile(0.9))

In [ ]:
g = sns.FacetGrid(with_result, height=8, aspect=2, xlim=(0,80), margin_titles=True, despine=True)
#g.map(sns.distplot, "ind_error", hist_kws={'alpha':0.8}).add_legend()
sns.distplot(with_result.ind_error, bins=250, kde=True)

In [ ]:
with_result["capped_num_runs"] = np.clip(with_result.num_runs, 1, shared.num_pace_years + 1)


In [ ]:
g = sns.FacetGrid(with_result, hue="capped_num_runs", height=8, aspect=2, xlim=(0,80), margin_titles=True, despine=True)
g.map(sns.distplot, "ind_error", hist=False, hist_kws={'alpha':0.8}).add_legend()

In [ ]:
def calculate_relay_values(result_row):
    rows_so_far = with_result[(with_result.team_id == result_row.team_id) & (with_result["leg"] <= result_row["leg"])]

    return pd.Series({"real_relay_time": np.sum(rows_so_far["leg-time"]), "mean_num_runs": np.mean(rows_so_far["num_runs"]).round(0)})

relay_values = with_result.apply(lambda row: calculate_relay_values(row), axis=1)

with_result["real_relay_time"] = relay_values["real_relay_time"]
with_result["mean_num_runs"] = relay_values["mean_num_runs"]



In [ ]:
with_result["relay_error"] = np.abs(np.exp(with_result["fin_sum_log_mean"]) - with_result["real_relay_time"])
with_result["relay_error_ratio"] = with_result["relay_error"] / with_result["real_relay_time"]

In [ ]:
display(np.mean(with_result["relay_error"]))
display(np.mean(with_result["relay_error_ratio"]))

In [ ]:
reports.append(f'Viestiennusteen keskivirhe: {np.mean(with_result["relay_error"]).round(3)} minuuttia')  
reports.append(f'Viestiennusteen virheen mediaani: {np.median(with_result["relay_error"]).round(3)} minuuttia')  
display(reports)               

In [ ]:
reports.append(with_result[["leg", 'relay_error']].groupby('leg').agg(["mean", "median", "count"]).round(2).to_string())
display(reports)

In [ ]:
plt.figure(figsize=(16, 10))
plot = sns.scatterplot(x="team_id", y="relay_error", hue="leg", data=with_result)
plot.axes.set_ylim(0, with_result.relay_error.quantile(0.95))


In [ ]:
plt.figure(figsize=(16, 10))
plot = sns.scatterplot(x="team_id", y="relay_error", hue="mean_num_runs", data=with_result)
plot.axes.set_ylim(0, with_result.relay_error.quantile(0.95))

In [ ]:
g = sns.FacetGrid(with_result, hue="leg", height=8, aspect=2, xlim=(0,180), margin_titles=True, despine=True)
sns.distplot(with_result.relay_error, bins=250, kde=True)

In [ ]:
g = sns.FacetGrid(with_result, hue="leg", height=8, aspect=2, xlim=(0,180), margin_titles=True, despine=True)
g.map(sns.distplot, "relay_error", hist=False, hist_kws={'alpha':0.8}).add_legend()

In [ ]:
with_result["real_finish_time"] = pd.to_timedelta(with_result["real_relay_time"], unit="min") + shared.start_timestamp[ve_or_ju][year]

with_result["real_interval_error"] = (with_result["real_finish_time"] < with_result["fin_time_start95"]) | (with_result["real_finish_time"] > with_result["fin_time_end95"])
with_result["real_interval_error_fast"] = (with_result["real_finish_time"] < with_result["fin_time_start95"])
with_result["real_interval_error_slow"] = (with_result["real_finish_time"] > with_result["fin_time_end95"])
np.mean(with_result["real_interval_error"])

reports.append(f'Viestin aikaväliennuste väärin: {np.mean(with_result["real_interval_error"]).round(3) * 100} %')
display(reports)

In [ ]:
display(np.mean(with_result["real_interval_error_fast"]))
display(np.mean(with_result["real_interval_error_slow"]))



In [ ]:
with_result["real_interval"] = with_result["fin_time_end95"] - with_result["fin_time_start95"]
with_result["real_interval"] =  pd.to_timedelta(with_result["real_interval"].values).total_seconds() / 60


In [ ]:




reports.append(with_result[["leg", 'real_interval', 'real_interval_error', 'real_interval_error_fast', 'real_interval_error_slow']].groupby('leg').agg(["mean"]).round(3).to_string())
display(reports)

In [ ]:
by_num_runs = with_result[["mean_num_runs", "real_interval", 'real_interval_error', 'real_interval_error_fast', 'real_interval_error_slow']].groupby('mean_num_runs').agg(["mean"]).round(3)
reports.append(by_num_runs.to_string())
by_num_runs

In [ ]:
shared.write_simple_text_report(reports, f'post_race_analysis_{ve_or_ju}.txt')

In [ ]:
#out_of_interval = with_result[with_result["real_interval_error"]]
plt.figure(figsize=(16, 10))
plot = sns.scatterplot(x="team_id", y="real_relay_time", hue="real_interval_error", alpha=0.3, data=with_result)
plot.axes.set_ylim(with_result.real_relay_time.min() - 5, with_result.real_relay_time.quantile(0.995))

In [ ]:
g = sns.FacetGrid(with_result, row="leg", hue="real_interval_error", xlim=(0,with_result.team_id.max()), height=6, aspect=1, legend_out=False)
g.map(sns.regplot, "team_id", "fin_sum_log_std", scatter_kws={'alpha':0.1}, order=2).add_legend()


In [ ]:

shared.log_df(f"{ve_or_ju} DONE")